In [ ]:
!mkdir -p ~/agave/aaron

%cd ~/agave/aaron

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *

!auth-tokens-refresh

In [ ]:
!echo ${AGAVE_STORAGE_SYSTEM_ID}

In [ ]:
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N /home/jovyan/funwave/
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N /home/jovyan/funwave/build

In [ ]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout -b dev"
writefile("version.txt","""3
2
0
dev""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F version.txt /home/jovyan/funwave

Build wrapper now clones FWV too

In [ ]:
writefile("funwave-build-wrapper.txt","""

VERSION=$(cat version.txt | paste -sd "." -)

git clone https://github.com/fengyanshi/FUNWAVE-TVD
git checkout -b dev

sudo docker build \
    --build-arg "BUILD_DATE=\${AGAVE_JOB_SUBMIT_TIME}" \
    --build-arg "VERSION=\${VERSION}" \
    --rm -t funwave-tvd:\${VERSION} .

docker inspect funwave-tvd:\${VERSION}

""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-build-wrapper.txt /home/jovyan/funwave/build

In [ ]:
writefile("Dockerfile","""
FROM stevenrbrandt/science-base
MAINTAINER Steven R. Brandt <sbrandt@cct.lsu.edu>

ARG BUILD_DATE
ARG VERSION


LABEL org.agaveplatform.ax.architecture="x86_64"                                \
      org.agaveplatform.ax.build-date="\$BUILD_DATE"                             \
      org.agaveplatform.ax.version="\$VERSION"                             \
      org.agaveplatform.ax.name="${AGAVE_USERNAME}/funwave-tvd"    \
      org.agaveplatform.ax.summary="Funwave-TVD is a code to simulate the shallow water and Boussinesq equations written by Dr. Fengyan Shi." \
      org.agaveplatform.ax.vcs-type="git"                                       \
      org.agaveplatform.ax.vcs-url="https://github.com/fengyanshi/FUNWAVE-TVD" \
      org.agaveplatform.ax.license="BSD 3-clause"
      
USER root
RUN mkdir -p /home/install/FUNWAVE-TVD
RUN chown jovyan /home/install/FUNWAVE-TVD
RUN pwd
COPY --chown=jovyan:jovyan ./FUNWAVE-TVD /home/install/FUNWAVE-TVD
RUN ls /home/install
RUN ls /home/install/FUNWAVE-TVD
USER jovyan

WORKDIR /home/install/FUNWAVE-TVD/src
RUN perl -p -i -e 's/FLAG_8 = -DCOUPLING/#$&/' Makefile && \
    make

RUN mkdir -p /home/jovyan/rundir
WORKDIR /home/jovyan/rundir
""")
#!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N /home/jovyan/FUNWAVE-TVD/build/
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F Dockerfile /home/jovyan/funwave

In [ ]:
writefile("funwave-build-app.txt","""
{  
   "name":"${AGAVE_USERNAME}-${MACHINE_NAME}-funwave-dbuild",
   "version":"1.0",
   "label":"Builds the funwave docker image",
   "shortDescription":"Funwave docker build",
   "longDescription":"",
   "deploymentSystem":"${AGAVE_STORAGE_SYSTEM_ID}",
   "deploymentPath":"funwave/",
   "templatePath":"build/funwave-build-wrapper.txt",
   "testPath":"version.txt",
   "executionSystem":"${AGAVE_EXECUTION_SYSTEM_ID}",
   "executionType":"CLI",
   "parallelism":"SERIAL",
   "modules":[],
   "inputs":[],
   "parameters":[{
     "id" : "code_version",
     "value" : {
       "visible":true,
       "required":true,
       "type":"string",
       "order":0,
       "enquote":false,
       "default":"latest"
     },
     "details":{
         "label": "Version of the code",
         "description": "If true, output will be packed and compressed",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     },
     "semantics":{
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     }
   }],
   "outputs":[]
}
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-build-app.txt /home/jovyan/funwave/build

In [ ]:
!pwd
!ls

In [ ]:
!apps-addupdate -F funwave-build-app.txt

In [ ]:
requestbin_url = !requestbin-create
os.environ['REQUESTBIN_URL'] = requestbin_url[0]
setvar("""
WEBHOOK_URL=${REQUESTBIN_URL}
""")

In [ ]:
!pwd
writefile("funwave-build-job.json","""
 {
   "name":"funwave-build",
   "appId": "${AGAVE_USERNAME}-${MACHINE_NAME}-funwave-dbuild-1.0",
   "maxRunTime":"00:10:00",
   "archive": false,
   "notifications": [
    {
      "url":"${WEBHOOK_URL}",
      "event":"*",
      "persistent":"true"
    }
   ],
   "parameters": {
     "code_version":"latest"
   }
 }
""")

In [ ]:
setvar("""
# Capture the output of the job submit command
OUTPUT=$(jobs-submit -F funwave-build-job.json)
# Parse out the job id from the output
JOB_ID=$(echo $OUTPUT | cut -d' ' -f4)
""")

In [ ]:
from time import sleep
for iter in range(25):
    setvar("STAT=$(jobs-status $JOB_ID)")
    stat = os.environ["STAT"]
    sleep(5.0)
    if stat == "FINISHED" or stat == "FAILED":
        break

In [ ]:
!jobs-output-get -P ${JOB_ID} funwave-build.out